In [1]:
%config Completer.use_jedi = False

In [110]:
import numpy as np
import pandas as pd
from scipy.stats import beta, bernoulli
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from plotly import tools
import random
import math

RANDOM_SEED = 123
np.random.seed(RANDOM_SEED)
init_notebook_mode(connected=True)

In [111]:
# Unreallistic
actual_ctr = [.45, .65]

In [214]:
def algorithm_performance():
    """
    Function that will show the performance of each algorithm we will be using in this tutorial.
    """
    
    ## calculate how many time each Ad has been choosen
    count_series = pd.Series(index_list).value_counts(normalize=True)
    print('Ad #0 has been shown', count_series[0]*100, '% of the time.')
    print('Ad #1 has been shown', count_series[1]*100, '% of the time.')
    
    print('Total Reward (Number of Clicks):', total_reward) ## print total Reward
    
    x = np.arange (0, n, 1)

    ## plot the calculated CTR for Ad #0
    data1 = go.Scatter(x=x,
                       y=ctr[0],
                       name='Calculated CTR #0',
                       line=dict(color=('rgba(10, 108, 94, .7)'),
                                 width=2))

    ## plot the line with actual CTR for Ad #0
    data2 = go.Scatter(x=[0, n],
                       y=[actual_ctr[0]] * 2,
                       name='Actual CTR #0 value',
                       line = dict(color = ('rgb(205, 12, 24)'),
                                   width = 1,
                                   dash = 'dash'))

    ## plot the calculated CTR for Ad #1
    data3 = go.Scatter(x=x,
                       y=ctr[1],
                       name='Calculated CTR #1',
                       line=dict(color=('rgba(187, 121, 24, .7)'),
                                 width=2))

    ## plot the line with actual CTR for Ad #0
    data4 = go.Scatter(x=[0, n],
                       y=[actual_ctr[1]] * 2,
                       name='Actual CTR #1 value',
                       line = dict(color = ('rgb(205, 12, 24)'),
                                   width = 1,
                                   dash = 'dash'))

    ## plot the Regret values as a function of trial number
    data5 = go.Scatter(x=x,
                       y=regret_list,
                       name='Regret')

    layout = go.Layout(title='Simulated CTR Values and Algorithm Regret',
                       xaxis={'title': 'Trial Number'},
                       yaxis1={'title': 'CTR value'},
                       yaxis2={'title': 'Regret Value'}
                       )
    fig = tools.make_subplots(rows=2, cols=1, print_grid=False, shared_yaxes=True, shared_xaxes=True)

    fig.append_trace(data1, 1, 1)
    fig.append_trace(data2, 1, 1)
    fig.append_trace(data3, 1, 1)
    fig.append_trace(data4, 1, 1)
    fig.append_trace(data5, 2, 1)

    fig['layout'].update(layout)
    iplot(fig, show_link=False)

### Random Selection
- 0% = Exloration
- 100% = Exploitation

In [215]:
n = 1000 

In [216]:
total_reward

636

In [217]:
regret = 0
total_reward = 0
regret_list = []
ctr = {0: [], 1:[]} #lists for collecting the calculated CTR
index_list = [] # lists for collecting the number of randomly choosen Ad

#initial values for impressons and clicks
impressions = [0,0]
clicks = [0,0]

for i in range(n):
    random_index = np.random.randint(0,2,1)[0] # randomly choose the value between [0,1]
    index_list.append(random_index)
    impressions[random_index] += 1
    did_click = bernoulli.rvs(actual_ctr[random_index])
    
    if did_click:
        clicks[random_index] += did_click
    
    if impressions[0] == 0 :
        ctr_0 = 0
    
    else :
         ctr_0 = clicks[0]/impressions[0]
    
    if impressions[1] == 0:
        ctr_1 = 0
    else :
        ctr_1 = clicks[1]/impressions[1]
    
    ctr[0].append(ctr_0)
    ctr[1].append(ctr_1)
    
    ## calculate the regret and reward
    regret += max(actual_ctr) - actual_ctr[random_index]
    regret_list.append(regret)
    total_reward += did_click

In [218]:
algorithm_performance()

Ad #0 has been shown 48.4 % of the time.
Ad #1 has been shown 51.6 % of the time.
Total Reward (Number of Clicks): 546


regret이 계속 증가하는 모습이다.  
더 좋은 알고리즘으로 이 regret을 줄이는 것이 우리의 목표가 되겠다. 


In [219]:
random_dict = {
    'reward' : total_reward,
    'regret_list' : regret_list,
    'ads_count' : pd.Series(index_list).value_counts(normalize=True)
}

## Epsilon Greedy
- ~15%까지는 Exploration
- ~85%까지 Exploitation

1. 초기 몇번 까지는 Exploration
2. 각 Exploration마다 최고 점수를 받는 variant 고르기
3. Epsilon 설정
4. (1-E)%의 winning variant를 고르고 다른 옵션에는 E%를 설정한다.

In [220]:
e = 0.05
n_init = 100
impressions = [0,0]
clicks = [0,0]

for i in range(n_init):
    random_index = np.random.randint(0,2,1)[0]
    
    impressions[random_index] += 1
    did_click = bernoulli.rvs(actual_ctr[random_index])
    if did_click:
        clicks[random_index] += did_click
    
ctr_0 = clicks[0] / impressions[0]
ctr_1 = clicks[1] / impressions[1]
win_index = np.argmax([ctr_0, ctr_1])

print('After', n_init, 'initial trials Ad #', \
      win_index, 'got the highest CTR', round(np.max([ctr_0, ctr_1]),2),
     '(Real CTR value is', actual_ctr[win_index], ')'
      '\nIt will be shown', (1-e)*100, '% of the time.')

After 100 initial trials Ad # 1 got the highest CTR 0.57 (Real CTR value is 0.65 )
It will be shown 95.0 % of the time.


In [221]:
regret = 0
total_reward = 0
regret_list = []
ctr = {0 : [], 1: []}
index_list = []
impressions = [0,0]
clicks = [0,0]

for i in range(n):
    epsilon_index = random.choices([win_index, 1-win_index],
                                  [1-e, e])[0]
    index_list.append(epsilon_index)
    
    impressions[epsilon_index] +=1
    did_click = bernoulli.rvs(actual_ctr[epsilon_index])
    if did_click :
        clicks[epsilon_index] += did_click
        
    if impressions[0] == 0 :
        ctr_0 = 0
    else :
        ctr_0 = clicks[0]/impressions[0]
        
    if impressions[1] ==0 :
        ctr_1 = 0
    else :
        ctr_1 = clicks[1]/impressions[1]
        
    ctr[0].append(ctr_0)
    ctr[1].append(ctr_1)
    
    regret += max(actual_ctr) - actual_ctr[epsilon_index]
    regret_list.append(regret)
    total_reward += did_click

In [222]:
algorithm_performance()

Ad #0 has been shown 6.2 % of the time.
Ad #1 has been shown 93.8 % of the time.
Total Reward (Number of Clicks): 642


Random Selection model보다는 훨씬 괜찮은 결과가 나왔다. 하지만 탐색시의 winning variant는 최적의 variant가 아니다. 사실 suboptimal variant로 exploit 한 것이다. 이것은 regret을 올리고 보상을 감소시킨다. 큰 수의 법칙에 따라 초기 시도를 많이 할수록, winning variant를 찾을 가능성이 크다. 하지만 마케팅에서는 큰 수의 법칙에 따르고 싶지 않을 것이다. 그럴만한 예산이 없을 거니까...

> 좋은 점은 어떤 비율을 설정할 수 있다는 것이다. 다른 epsilon값을 선택함으로써 얼마나 자주 winning ad를 보여줄 수 있는지 조정할 수 있다.

In [223]:
epsilon_dict = {
    'reward' : total_reward,
    'regret_list' : regret_list,
    'ads_count' : pd.Series(index_list).value_counts(normalize=True)
}

## Thompson Sampling
- 50% Exploration
- 50% Exploitation

톰슨 샘플링의 탐색 부분은 e-greedy알고리즘보다 복잡하다. Beta distribution을 이용한다. 왜냐하면 광고를 클릭하는 것은 베르누의 과정에 속하기 때문이다.(클릭했다, 안했다, 1,0) 하지만 톰슨 샘플링은 일반적으로 어떤 분포, 어떤 파라미터에서든지 샘플링이 가능하다.

- Beta 분포는 alpha와 beta 파라미터로 분포의 모양을 조절한다.

로직은 다음과 같다.
1. alpha와 beta를 고른다.
2. $\alpha = prior + hits$, $\beta = prior + misses$로 계산한다. 우리의 경우는 hits는 클릭 수를 말하고, misses는 클릭없이 impression된 경우를 말한다(클릭 없는 노출). prior는 CTR에 대한prior정보가 있으면 유용하다. 우리는 갖고 있지 않으므로 1.0을 사용한다.
3. CTR을 추정한다. 실제 CTR을 베타 분포에서 샘플링하고 ($B(\alpha_i,\beta_i)$)에서, 추정 CTR이 가장 높은 것을 선택한다.
4. 2-3을 반복한다.

In [224]:
regret = 0
total_reward = 0
regret_list = []
ctr = {0 : [], 1: []}
index_list = []
impressions = [0,0]
clicks = [0,0]
priors = (1,1)
#randomly choose the first shown ad
win_index = np.random.randint(0,2,1)[0] 

for i in range(n):
    impressions[win_index] += 1
    did_click = bernoulli.rvs(actual_ctr[win_index])
    if did_click :
        clicks[win_index] += did_click
    
    ctr_0 = random.betavariate(priors[0] + clicks[0], priors[1] + impressions[0] - clicks[0])
    ctr_1 = random.betavariate(priors[1] + clicks[1], priors[1] + impressions[1] - clicks[1])
    
    win_index = np.argmax([ctr_0, ctr_1])
    index_list.append(win_index)
    
    ctr[0].append(ctr_0)
    ctr[1].append(ctr_1)
    
    regret += max(actual_ctr) - actual_ctr[win_index]
    regret_list.append(regret)
    total_reward += did_click

In [225]:
## plot the Beta distributions
x = np.arange (0, 1, 0.01)
y = beta.pdf(x, priors[0]+clicks[0], priors[1] + impressions[0] - clicks[0])
y /= y.max() ## normalize

data1 = go.Scatter(x=x,
                   y=y,
                   name='(Ad #0)',
                   marker = dict(color=('rgba(10, 108, 94, 1)')),
                   fill='tozeroy',
                   fillcolor = 'rgba(10, 108, 94, .7)')

data2 = go.Scatter(x = [actual_ctr[0]] * 2,
                   y = [0, 1],
                   name = 'Actual CTR #0 Value',
                   mode='lines',
                   line = dict(
                       color = ('rgb(205, 12, 24)'),
                       width = 2,
                       dash = 'dash'))

y = beta.pdf(x, priors[0]+clicks[1], priors[1] + impressions[1] - clicks[1])
y /= y.max()

data3 = go.Scatter(x=x,
                   y=y,
                   name='(Ad #1)',
                   marker = dict(color=('rgba(187, 121, 24, 1)')),
                   fill='tozeroy',
                   fillcolor = 'rgba(187, 121, 24, .7)')

data4 = go.Scatter(x = [actual_ctr[1]] * 2,
                   y = [0, 1],
                   name = 'Actual CTR #1 Value',
                   mode='lines',
                   line = dict(
                       color = ('rgb(205, 12, 24)'),
                       width = 2,
                       dash = 'dash'))

layout = go.Layout(title='Beta Distributions for both Ads',
                   xaxis={'title': 'Possible CTR values'},
                   yaxis={'title': 'Probability Density'})

fig = go.Figure(data=[data1, data2, data3, data4], layout=layout)

# fig = tools.make_subplots(rows=1, cols=2, print_grid=False, shared_xaxes=False,
#                           subplot_titles=('Beta Distribution (Ad #0)','Beta Distribution (Ad #1)'))

# fig.append_trace(data1, 1, 1)
# fig.append_trace(data2, 1, 1)
# fig.append_trace(data3, 1, 2)
# fig.append_trace(data4, 1, 2)

# fig['layout'].update(showlegend=False)

iplot(fig, show_link=False)

겹쳐지는 영역을 보자. Ad0의 beta distribution 값이 더 클 것이다.

In [226]:
algorithm_performance()

Ad #0 has been shown 4.2 % of the time.
Ad #1 has been shown 95.8 % of the time.
Total Reward (Number of Clicks): 647


지금까지 본 regret중 가장 낮은 regret을 확인할 수 있다. CTR 플롯에서, 시작점 부분을 보면, 초록색 AD0번이 Ad 1버ㄴ보다 높은 것을 볼 수 있다. 

알고리즘은 지속적으로 탐색한다. 자연스럽게 beta distribution을 이용해 가장 가치가 높은 샘플을 가져와서 exploit한다. Beta distribution ad1은 더 높고 좁은 분포를 갖고 있다. 이것은 샘플된 값들이 항상 ad0보다 높을 것이라는 것을 의미한다. 결국 ad1이 우리가 원하는 광고이다.

In [227]:
thompson_dict = {'reward':total_reward, 
                 'regret_list':regret_list, 
                 'ads_count':pd.Series(index_list).value_counts(normalize=True)}

## Upper Confidence Bound(UCB1)
- 50% Exploration
- 50% Exploitation

톰슨 샘플링과 달리 UCB는 불확실성에 더 초점을 맞춘다. 한 variant에 대해 더 불확실 할 수록, 더 탐색을 해야만 한다.

알고리즘은 variant를 UCB로 고른다. UCB는 가장 높은 보상이 나올 것이라고 추측되는 variant를 나타낸다. 
$$UCB = \bar x_i + \sqrt{\frac{2 \cdot \log{t}}{n}}$$

$\bar{x}_i$ CTR이 i번째 단계일때,  
$t$ - 모든 변량들을 다 더한 숫자이다.  
$n$ - 선택된 변량에 대해 다 더해진 숫자이다.

로직은 직관적이다.
1. UCB를 모든 변량들에 대해 구한다.
2. 가장 높은 UCB를 가진 변량을 선택한다.
3. 1번으로 다시 돌아간다.

In [228]:
regret = 0 
total_reward = 0
regret_list = [] 
index_list = [] 
impressions = [0,0] 
clicks = [0,0]
ctr = {0: [], 1: []}
total_reward = 0

for i in range(n):
    
    index = 0
    max_upper_bound = 0
    for k in [0,1]:
        if (impressions[k] > 0):
            CTR = clicks[k] / impressions[k]
            delta = math.sqrt(2 * math.log(i+1) / impressions[k])
            upper_bound = CTR + delta
            ctr[k].append(CTR)
        else:
            upper_bound = 1e400
        if upper_bound > max_upper_bound:
            max_upper_bound = upper_bound
            index = k
    index_list.append(index)
    impressions[index] += 1
    reward = bernoulli.rvs(actual_ctr[index])
    
    clicks[index] += reward
    total_reward += reward
    
    regret += max(actual_ctr) - actual_ctr[index]
    regret_list.append(regret)

In [229]:
algorithm_performance()

Ad #0 has been shown 19.2 % of the time.
Ad #1 has been shown 80.80000000000001 % of the time.
Total Reward (Number of Clicks): 596


In [230]:
ucb1_dict = {'reward':total_reward, 
             'regret_list':regret_list, 
             'ads_count':pd.Series(index_list).value_counts(normalize=True)}

### 결론 및 성능 비교

In [231]:
data1 = go.Bar(x=['Random Selection', 'Epsilon Greedy', 'Thompson Sampling', 'UCB1'],
               y=[random_dict['ads_count'][0], 
                  epsilon_dict['ads_count'][0], 
                  thompson_dict['ads_count'][0],
                  ucb1_dict['ads_count'][0]],
               name='Ad #0',
               marker=dict(color='rgba(10, 108, 94, .7)'))

data2 = go.Bar(x=['Random Selection', 'Epsilon Greedy', 'Thompson Sampling', 'UCB1'],
               y=[random_dict['ads_count'][1], 
                  epsilon_dict['ads_count'][1], 
                  thompson_dict['ads_count'][1],
                  ucb1_dict['ads_count'][1]],
               name='Ad #1',
               marker=dict(color='rgba(187, 121, 24, .7)'))

data = [data1, data2]
layout = go.Layout(title='Ratio of appearance of both Ads throughout the trials',
                   xaxis={'title': 'Algorithm'},
                   yaxis={'title': 'Ratio'},
                   barmode='stack')

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [232]:
data1 = go.Scatter(
    x=np.arange (0, n, 1),
    y=random_dict['regret_list'],
    name='Random Selection',
    marker=dict(color='#ffcc66')
)
data2 = go.Scatter(
    x=np.arange (0, n, 1),
    y=epsilon_dict['regret_list'],
    name='e-Greedy',
    marker=dict(color='#0099ff')
)
data3 = go.Scatter(
    x=np.arange (0, n, 1),
    y=thompson_dict['regret_list'],
    name='Thompson Sampling',
    marker=dict(color='#ff3300')
)
data4 = go.Scatter(
    x=np.arange (0, n, 1),
    y=ucb1_dict['regret_list'],
    name='UCB1',
    marker=dict(color='#33cc33')
)

layout = go.Layout(
    title='Regret by the Algorithm',
    xaxis={'title': 'Trial'},
    yaxis={'title': 'Regret'}
)

data = [data1, data2, data3, data4]
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [233]:
data = go.Bar(
    x=[ucb1_dict['reward'], thompson_dict['reward'], epsilon_dict['reward'], random_dict['reward']],
    y=['UCB1', 'Thompson Sampling', 'e-Greedy','Random Selection'],
    orientation = 'h',
    marker=dict(color=['#33cc33', '#ff3300', '#0099ff', '#ffcc66']),
    opacity=0.7
)

text = go.Scatter(
    x=[ucb1_dict['reward'], thompson_dict['reward'], epsilon_dict['reward'], random_dict['reward']],
    y=['UCB1', 'Thompson Sampling', 'e-Greedy', 'Random Selection'],
    mode='text',
    text=[ucb1_dict['reward'], thompson_dict['reward'], epsilon_dict['reward'], random_dict['reward']],
    textposition='middle left',
    line = dict(
        color = ('rgba(255,141,41,0.6)'),
        width = 1
    ),
    textfont=dict(
        family='sans serif',
        size=16,
        color='#000000'
    )
)

data = [data,text]

layout = go.Layout(
    title='Total Reward by Algorithms',
    xaxis={'title': 'Total Reward (Clicks)'},
    margin={'l':200},
    showlegend=False
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)